In [1]:
## run notebooks
%run "../Notebooks/intialization.ipynb"
%run "../Notebooks/Stitching_functions.ipynb"
%run "../Notebooks/Visualization_functions.ipynb"
%run "../Notebooks/Downsampling_and_outliers_functions.ipynb"
%load_ext autoreload

In [2]:
## parameters initialization 
myparams = "./ScreenCamera_2021-01-11-12-06-34.json"  # parameter for camera point view, json file via pressing P
myconfiguration_file = "RenderOption_2021-01-11-12-22-53.json" # configuration file for properties, json file via pressing o

In [45]:
#load point cloud
## TODO load the clean one 
pcd= o3d.io.read_point_cloud("complete_cluster_3.pcd")

In [50]:

custom_draw_geometry(pcd,
                     mytitle = "complete",
                     params = myparams,  # parameter for camera point view, json file via pressing P
                     configuration_file = myconfiguration_file, # configuration file for properties, json file via pressing o
                     take_screen_shot = False,
                     rotate = True)

loading parameters: 
  ./ScreenCamera_2021-01-11-12-06-34.json
loading configuration file: 
  RenderOption_2021-01-11-12-22-53.json
[Open3D WARNING] GLFW Error: The GLFW library is not initialized


## alpha mesh

In [62]:
alpha = 0.2
print(f"alpha={alpha:.3f}")
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha)
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)

alpha=0.200


## ball pivoting

In [15]:
radii = [0.005, 0.01, 0.02, 0.04]
rec_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    pcd, o3d.utility.DoubleVector(radii))
o3d.visualization.draw_geometries([pcd, rec_mesh], mesh_show_back_face=True)

## Poisson

In [47]:
print('run Poisson surface reconstruction')
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        pcd, depth=10)
print(mesh)
o3d.visualization.draw_geometries([pcd,mesh], mesh_show_back_face=True)

run Poisson surface reconstruction
[Open3D DEBUG] Input Points / Samples: 488502 / 356274
[Open3D DEBUG] #   Got kernel density: 0.180398 (s), 1184.3 (MB) / 1403.2 (MB) / 1881 (MB)
[Open3D DEBUG] #     Got normal field: 1.17449 (s), 1292.39 (MB) / 1403.2 (MB) / 1881 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 2.412229e-06 / 1.178379e+00
[Open3D DEBUG] #       Finalized tree: 0.904082 (s), 1494.27 (MB) / 1494.27 (MB) / 1881 (MB)
[Open3D DEBUG] #  Set FEM constraints: 1.299 (s), 1538.7 (MB) / 1538.7 (MB) / 1881 (MB)
[Open3D DEBUG] #Set point constraints: 0.242631 (s), 1518.7 (MB) / 1538.7 (MB) / 1881 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 5065145 / 5451904 / 336833
[Open3D DEBUG] Memory Usage: 1518.699 MB
[Open3D DEBUG] # Linear system solved: 2.69203 (s), 1518.95 (MB) / 1538.7 (MB) / 1881 (MB)
[Open3D DEBUG] Got average: 0.0613511 (s), 1392.09 (MB) / 1538.7 (MB) / 1881 (MB)
[Open3D DEBUG] Iso-Value: 4.994245e-01 = 2.439699e+05 / 4.885020e+05
[Open3D DEBUG] #

In [48]:
print('visualize densities')
densities = np.asarray(densities)
density_colors = plt.get_cmap('plasma')(
    (densities - densities.min()) / (densities.max() - densities.min()))
density_colors = density_colors[:, :3]
density_mesh = o3d.geometry.TriangleMesh()
density_mesh.vertices = mesh.vertices
density_mesh.triangles = mesh.triangles
density_mesh.triangle_normals = mesh.triangle_normals
density_mesh.vertex_colors = o3d.utility.Vector3dVector(density_colors)
o3d.visualization.draw_geometries([pcd,density_mesh], mesh_show_back_face=True)

visualize densities


In [49]:
print('remove low density vertices')
vertices_to_remove = densities < np.quantile(densities, 0.01)
mesh.remove_vertices_by_mask(vertices_to_remove)
print(mesh)
o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)

remove low density vertices
TriangleMesh with 1007625 points and 2013402 triangles.


In [30]:
o3d.visualization.draw_geometries([pcd,density_mesh], mesh_show_back_face=True)

In [28]:
custom_draw_geometry([pcd,density_mesh],
                     mytitle = "complete",
                     params = myparams,  # parameter for camera point view, json file via pressing P
                     configuration_file = myconfiguration_file, # configuration file for properties, json file via pressing o
                     take_screen_shot = False,
                     rotate = True)

loading parameters: 
  ./ScreenCamera_2021-01-11-12-06-34.json
loading configuration file: 
  RenderOption_2021-01-11-12-22-53.json
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
loading parameters: 
  ./ScreenCamera_2021-01-11-12-06-34.json
loading configuration file: 
  RenderOption_2021-01-11-12-22-53.json
[Open3D WARNING] GLFW Error: The GLFW library is not initialized


In [11]:
## surface normal estimation 
print("Computing normal and rendering it.")
mesh.compute_vertex_normals()
print(np.asarray(mesh.triangle_normals))
o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)

Computing normal and rendering it.
[[ 0.33686456 -0.48890054  0.80467293]
 [ 0.32182805 -0.53043395  0.78426178]
 [ 0.36669547 -0.54204341  0.75612392]
 ...
 [-0.69944585 -0.682177    0.21309632]
 [-0.88240685 -0.46964167  0.02819312]
 [-0.76159627 -0.64595258  0.0521189 ]]


## filters

In [63]:
#this looks like the best
mesh_in = mesh
print('filter with average with 1 iteration')
mesh_out = mesh_in.filter_smooth_simple(number_of_iterations=1)
mesh_out.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh_out], mesh_show_back_face=True)

print('filter with average with 5 iterations')
mesh_out = mesh_in.filter_smooth_simple(number_of_iterations=10)
mesh_out.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh_out], mesh_show_back_face=True)

filter with average with 1 iteration
filter with average with 5 iterations


In [64]:
print('filter with Laplacian with 10 iterations')
mesh_out = mesh_in.filter_smooth_laplacian(number_of_iterations=10)
mesh_out.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh_out],mesh_show_back_face=True)

print('filter with Laplacian with 50 iterations')
mesh_out = mesh_in.filter_smooth_laplacian(number_of_iterations=50)
mesh_out.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh_out], mesh_show_back_face=True)

filter with Laplacian with 10 iterations
filter with Laplacian with 50 iterations


In [55]:
print('filter with Taubin with 10 iterations')
mesh_out = mesh_in.filter_smooth_taubin(number_of_iterations=10)
mesh_out.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh_out])

print('filter with Taubin with 100 iterations')
mesh_out = mesh_in.filter_smooth_taubin(number_of_iterations=100)
mesh_out.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh_out])

filter with Taubin with 10 iterations
filter with Taubin with 100 iterations


## show structure of the mesh

In [57]:
mesh = mesh_out
mesh.compute_vertex_normals()
print(    f'The mesh has {len(mesh.vertices)} vertices and {len(mesh.triangles)} triangles' )

o3d.visualization.draw_geometries([mesh], mesh_show_wireframe=True)
mesh = mesh.subdivide_loop(number_of_iterations=1)
print(  f'After subdivision it has {len(mesh.vertices)} vertices and {len(mesh.triangles)} triangles')
o3d.visualization.draw_geometries([mesh], mesh_show_wireframe=True)

The mesh has 197516 vertices and 409036 triangles
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNIN

[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARN

[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARN

[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARN

[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARN

...

[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARN

[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARN

[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARN

[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARN

[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARNING] [SubdivideLoop] non-manifold edge.
[Open3D WARN

## connected components

In [58]:
print("connected components")
vert = np.asarray(mesh.vertices)
min_vert, max_vert = vert.min(axis=0), vert.max(axis=0)
for _ in range(30):
    cube = o3d.geometry.TriangleMesh.create_box()
    cube.scale(0.005, center=cube.get_center())
    cube.translate(
        (
            np.random.uniform(min_vert[0], max_vert[0]),
            np.random.uniform(min_vert[1], max_vert[1]),
            np.random.uniform(min_vert[2], max_vert[2]),
        ),
        relative=False,
    )
    mesh += cube
mesh.compute_vertex_normals()
print("Show input mesh")
o3d.visualization.draw_geometries([mesh])

connected components
Show input mesh


In [59]:
print("Cluster connected triangles")
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    triangle_clusters, cluster_n_triangles, cluster_area = (
        mesh.cluster_connected_triangles())
triangle_clusters = np.asarray(triangle_clusters)
cluster_n_triangles = np.asarray(cluster_n_triangles)
cluster_area = np.asarray(cluster_area)

Cluster connected triangles
[Open3D DEBUG] [ClusterConnectedTriangles] Compute triangle adjacency
[Open3D DEBUG] [ClusterConnectedTriangles] Done computing triangle adjacency
[Open3D DEBUG] [ClusterConnectedTriangles] Done clustering, #clusters=40


In [60]:
print("Show mesh with small clusters removed")
mesh_0 = copy.deepcopy(mesh)
triangles_to_remove = cluster_n_triangles[triangle_clusters] < 100
mesh_0.remove_triangles_by_mask(triangles_to_remove)
o3d.visualization.draw_geometries([mesh_0])

Show mesh with small clusters removed
